In [1]:
import numpy as np
import os
import torch
import torch.nn as nn
from tqdm import tqdm
from torchvision.datasets import CIFAR10, MNIST, FashionMNIST, SVHN
from torch.nn import CrossEntropyLoss
from torch.optim import SGD
from torch.utils.data import DataLoader,TensorDataset
import torch.nn.functional as F
from torchvision.transforms import ToTensor
import random
# from feature_wise_pgd import PGD
from resnet20 import ResNet20
from LeNet import LeNet5
import time

In [2]:
batch_size = 1

# test_dataset=TensorDataset(x_test,y_test)
train_dataset = SVHN(root='../data', split='train', transform=ToTensor(), download=True)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

dataset = 'schn'
model = 'lenet5'
testmodel = torch.load('./pretrained/svhn_lenet5_0.842.pt')
VS_idx = np.load('./deepfeature/'+ dataset + '-' + model +'-VS-5.npy')

Using downloaded and verified file: ../data/train_32x32.mat


In [3]:
class PGD():
    def __init__(self, model, eps=8/255,
                 alpha=2/255, steps=3, random_start=True):
        self.model = model.cuda()
        self.eps = eps
        self.alpha = alpha
        self.steps = steps
        self.random_start = random_start
        self._supported_mode = ['default', 'targeted']
        self._targeted = False
        self.device = torch.device("cuda")

    def forward(self, images, labels, VS):

        images = images.clone().detach().to(self.device)
        labels = labels.clone().detach().to(self.device)

        if self._targeted:
            target_labels = self._get_target_label(images, labels)


        loss = nn.MSELoss()

        adv_images = images.clone().detach()

        _, clean_feature_map = self.model(images)
        # print(clean_feature_map.shape)
        cost_array = []
        adv_feature_map_array = []
        adv_images_array = []


        self.fuzz_images = images.clone().cuda()
        self.fuzz_labels = labels.clone().cuda()
        self.fuzz_outputs = labels.clone().cuda()
        start_time = time.time()
        while time.time()-start_time < 1:
            adv_images = images.clone().detach()
            if self.random_start:
                # Starting at a uniformly random point
                adv_images = adv_images + torch.empty_like(adv_images).uniform_(-self.eps, self.eps)
                adv_images = torch.clamp(adv_images, min=0, max=1).detach()
            for _ in range(self.steps):
                adv_images.requires_grad = True
                outputs, adv_feature_map = self.model(adv_images)


                # Calculate loss
                # print(adv_feature_map[:,idx].shape)
                # cost = nn.functional.cosine_similarity(torch.flatten(adv_feature_map[:,idx],   1)
                                                    #   ,torch.flatten(clean_feature_map[:,idx], 1), dim=0)
                # cost = cost.mean()
                cost = loss(adv_feature_map[:,VS], clean_feature_map[:,VS])
                # cost = loss(outputs, labels)

                # Update adversarial images
                grad = torch.autograd.grad(cost, adv_images,
                                        retain_graph=False, create_graph=False)[0]

                adv_images = adv_images.detach() + self.alpha * grad.sign()
                delta = torch.clamp(adv_images - images, min=-self.eps, max=self.eps)
                adv_images = torch.clamp(images + delta, min=0, max=1).detach()
                self.fuzz_images = torch.cat((self.fuzz_images,adv_images),0)
                self.fuzz_labels = torch.cat((self.fuzz_labels,labels),0)
                self.fuzz_outputs = torch.cat((self.fuzz_outputs,torch.argmax(outputs,dim=1)),0)

        return self.fuzz_images[1:], self.fuzz_labels[1:], self.fuzz_outputs[1:]

#             cost_array.append(cost.cpu().detach())
#             outputs, adv_feature_map = self.model(adv_images)
#             adv_images_array.append(adv_images)
#             adv_feature_map_array.append(adv_feature_map[:,idx])
            

#         cost_array = np.array(cost_array)
        




In [4]:
attack = PGD(testmodel, eps=0.1, alpha=0.1/2, steps=3, random_start=True)
start_time = time.time()
iterator=iter(train_loader)
image, label = next(iterator)
fuzz_images = image.clone().cuda()
fuzz_labels = label.clone().cuda()
truth_label = label.clone().cuda()
print(VS_idx)
for _, (image, label) in enumerate(train_loader):

    fuzzer_images,ground_truth, fuzzer_labels = attack.forward(image,label,VS_idx)
    fuzz_images = torch.cat((fuzz_images,fuzzer_images),0)
    fuzz_labels = torch.cat((fuzz_labels,fuzzer_labels),0)
    truth_label = torch.cat((truth_label,ground_truth),0)
    print(len(fuzzer_images))
    if time.time()-start_time>300:
        break

[ 5  7 14 13  2  9  1  4 11  0]
321
336
330
330
333
336
351
354
357
360
354
354
357
351
354
357
372
423
357
351
363
357
360
363
363
357
357
363
369
360
360
363
366
360
366
360
363
360
363
357
363
360
363
363
363
360
363
366
363
360
360
360
360
360
357
363
360
360
357
363
366
363
363
357
366
360
360
360
363
363
363
363
366
360
363
360
369
360
360
360
366
366
360
360
366
363
363
360
366
360
360
357
363
363
366
360
363
363
360
360
366
360
363
363
369
399
384
366
372
363
366
360
366
357
357
360
366
360
357
360
363
360
360
357
366
360
360
360
366
357
360
360
366
357
363
357
360
360
360
363
366
360
360
360
366
360
360
369
360
360
360
363
363
360
369
363
363
360
360
366
357
360
357
360
357
357
360
363
360
357
357
363
360
357
357
360
357
354
360
366
363
360
363
369
420
480
483
483
480
483
483
483
483
480
483
483
483
483
483
483
483
480
483
483
483
483
483
483
483
483
480
483
483
483
483
483
480
483
483
483
483
480
483
483
483
480
483
483
483
483
480
483
483
480
483
483
483
426
420
414
372
363


In [ ]:
print(len(fuzz_images),len(fuzz_labels))
# fuzz_labels = np.array(fuzz_labels)
# fuzz_images = np.array(fuzz_images)
# fuzz_x = np.concatenate(fuzz_images,axis = 0)
# fuzz_x = np.squeeze(fuzz_x,axis = 1)
print(fuzz_images.shape)
# fuzz_predict = np.concatenate(fuzz_labels,axis = 0)
print(fuzz_labels.shape)
# fuzz_predict = np.squeeze(fuzz_predict,axis = 1)
# print(fuzz_predict.shape)
# print(ground_truth.shape)
# fuzz_y = np.array(truth_label)
# fuzz_y = np.concatenate(truth_label,axis = 0)
print(truth_label.shape)

correct = truth_label.eq(fuzz_labels).sum().item()
# FDR = np.sum((fuzz_labels==truth_label))
fdd = {}
misclassfy_x = []
misclassfy_y = []
misclassify_idx = [] 
for i in range(len(fuzz_images)):
    if fuzz_labels[i]!=truth_label[i]:
        fdd[(str(fuzz_labels[i]),str(truth_label[i]))] = 1
        misclassify_idx.append(i)
print('Fuzzing number is:', correct)
print('Fuzzing fault type is:', len(fdd))
misclassfy_x = fuzz_images[misclassify_idx]
misclassfy_y = truth_label[misclassify_idx]
# np.save('./fuzz_cifar10_vgg16_images.npy', misclassfy_x.cpu().numpy())
# np.save('./fuzz_cifar10_vgg16_labels.npy', misclassfy_y.cpu().numpy())
print(misclassfy_x.shape)
print(misclassfy_y.shape)
# misclassfy=TensorDataset(misclassfy_x,misclassfy_y)
# torch.save(misclassfy, './rq5/cifar10_vgg16_fuzz.pt')

In [6]:
def train(model,train_loader,test_loader, optimizer):
    criterion = CrossEntropyLoss()

    for idx in range(2):
        with tqdm(train_loader[idx]) as loader:
            for data, target in loader:
                data, target = data.cuda(), target.cuda()
                optimizer.zero_grad()
                output,_ = model(data)
                loss = criterion(output, target)
                loss.backward()
                optimizer.step()

    all_correct_num = 0
    all_sample_num = 0
    for idx, (test_x, test_label) in enumerate(test_loader):
        test_x = test_x.cuda()
        predict_y, _ = model(test_x.float())
        predict_y = np.argmax(predict_y.cpu().detach(), axis=-1)
        # print(predict_y.shape)
        current_correct_num = predict_y == test_label
        all_correct_num += np.sum(current_correct_num.numpy(), axis=-1)
        all_sample_num += current_correct_num.shape[0]
    acc = all_correct_num / all_sample_num
    print('accuracy: {:.5f}'.format(acc))
#         scheduler.step()
    return acc
    

In [ ]:
from torch.utils.data import DataLoader,TensorDataset
testmodel = torch.load('./pretrained/svhn_lenet5_0.842.pt')
random_idx = np.random.choice(np.linspace(0, 37724, 37724, dtype=np.int32), 11273)
print(misclassfy_x.shape)
train_dataset=TensorDataset(misclassfy_x[torch.from_numpy(random_idx).type(torch.long)],misclassfy_y[torch.from_numpy(random_idx).type(torch.long)])
train_dataset_ori = SVHN(root='../data', split='train', transform=ToTensor(), download=True)
test_dataset = SVHN(root='../data', split='test', transform=ToTensor(), download=True)
train_loader_ori = DataLoader(train_dataset_ori, batch_size=128)
train_loader = DataLoader(train_dataset, batch_size=128)
test_loader = DataLoader(test_dataset, batch_size=512)

optimizer = torch.optim.SGD(testmodel.parameters(), lr=0.001, momentum=0.9, weight_decay=5e-4)
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, [14])
best_acc = 0
for e in range(20):
    print('Epoch: ', e+1)
    acc = train(testmodel, (train_loader, train_loader_ori), test_loader, optimizer)
    if acc > best_acc:
        best_acc = acc
        torch.save(model, './rq5/svhn_lenet5_best.pt')
    scheduler.step()
